In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import re

In [3]:
recipes = pd.read_csv('recipes.csv')
recipes.shape

(522517, 28)

In [8]:
recipes.head(2)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."


In [5]:
reviews = pd.read_csv('reviews.csv')
reviews.shape

(1401982, 8)

In [7]:
reviews.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


## Time

In [3]:
def convert_to_minutes(time_str):
    if pd.isnull(time_str):
        return None

    time_str = time_str.replace('PT', '')

    hours, minutes = 0, 0
    if 'H' in time_str:
        hours = int(time_str.split('H')[0])
        time_str = time_str.split('H')[1]

    if 'M' in time_str:
        minutes = int(time_str.split('M')[0])

    total_minutes = hours * 60 + minutes
    return total_minutes

recipes['CookTime'] = recipes['CookTime'].apply(convert_to_minutes)
recipes['PrepTime'] = recipes['PrepTime'].apply(convert_to_minutes)
recipes['TotalTime'] = recipes['TotalTime'].apply(convert_to_minutes)

In [4]:
recipes.sample(10)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
180043,188228,Baked Crunchy Trout,83907,bayousong,35.0,10,45,2006-09-27T19:09:00Z,"This is another quick, easy dish. We have trie...",character(0),Trout,"c(""< 60 Mins"", ""Easy"")","c(""2"", ""1"", ""8"", ""2 1/2"")","c(""trout fillets"", ""onion"")",NaN,NaN,351.2,15.0,2.6,131.5,119.0,3.8,0.5,1.6,47.4,4.0,NaN,"c(""Marinate fish filets for about 30 minutes.""..."
521296,540062,Perfect Flaky Pie Crust,2002627568,the baked angel,60.0,20,80,2020-04-10T19:57:00Z,Make and share this Perfect Flaky Pie Crust re...,"""https://img.sndimg.com/food/image/upload/w_55...",Pie,"c(""Dessert"", ""Vegan"", ""< 4 Hours"", ""Easy"")","c(""2 2/3"", ""1"", ""1"", ""6"")","c(""flour"", ""salt"", ""shortening"", ""cold water"")",NaN,NaN,504.3,34.7,8.6,0.0,389.1,42.4,1.5,0.1,5.7,6.0,1 pie pan,"c(""combine the flour, salt, and shortening int..."
367693,381061,Smoky Potatoes Au Gratin,1161666,Carol,65.0,30,95,2009-07-12T09:37:00Z,Make and share this Smoky Potatoes Au Gratin r...,character(0),Potato,"c(""Vegetable"", ""< 4 Hours"")","c(""8"", ""1/2"", ""2"", ""1/2"", ""1/4"", ""1/4"", ""1"", ""...","c(""margarine"", ""all-purpose flour"", ""low-fat m...",NaN,NaN,545.4,29.0,7.2,14.8,617.1,59.4,5.5,6.5,13.1,8.0,NaN,"c(""Preheat oven to 350 degrees."", ""Melt 8 tabl..."
174951,182976,Shish Kabob,274719,BurtonFanatic,15.0,1440,1455,2006-08-25T21:24:00Z,These lamb kabobs are marinated in very little...,character(0),Lamb/Sheep,"c(""Vegetable"", ""Meat"", ""Greek"", ""European"", ""S...","c(""2"", ""1/2"", ""1/4"", ""1"", ""1"", ""1/4"", ""1/2"", ""...","c(""olive oil"", ""chicken broth"", ""garlic"", ""sal...",4.5,2.0,224.6,10.8,3.6,72.6,211.0,5.3,1.2,3.1,26.4,8.0,NaN,"c(""Combine oil, broth, orange juice, lemon jui..."
174585,182598,Scones,176808,Pietro,30.0,20,50,2006-08-24T21:43:00Z,These are the little delights that grace the A...,"""https://img.sndimg.com/food/image/upload/w_55...",Scones,"c(""Quick Breads"", ""Breads"", ""Australian"", ""Hea...","c(""1 3/4"", NA, ""2"", ""30"", ""3/4"", NA, NA)","c(""self-raising flour"", ""salt"", ""sugar"", ""butt...",3.5,2.0,91.2,2.3,1.4,6.0,261.9,15.0,0.5,1.5,2.3,NaN,NaN,"c(""Preheat oven to 220 degrees celsius."", ""Com..."
439183,455433,Chocolate Mochi Recipe,930100,Queen Dana,55.0,0,55,2011-05-08T08:56:00Z,Make and share this Chocolate Mochi Recipe rec...,character(0),Dessert,"c(""Polynesian"", ""Healthy"", ""< 60 Mins"", ""Easy"")","c(""2"", ""2"", ""1"", ""1"", ""2"", ""1/2"", ""2"", ""2"")","c(""white sugar"", ""baking soda"", ""evaporated mi...",NaN,NaN,4996.2,161.3,74.8,569.3,5176.5,831.1,17.5,492.6,85.3,NaN,1 mochi,"""Sift dry ingredients, mochiko, sugar and baki..."
436838,453025,Colonel Sanders&rsquo; Hotwater Pie Crust,1873679,Colonel Sanders,NaN,45,45,2011-04-04T14:48:00Z,"This is my personal recipe for a delicious, fl...",character(0),Dessert,"c(""Grains"", ""< 60 Mins"", ""For Large Groups"", ""...","c(""1 1/4"", ""5"", ""1"", ""1"", ""1"", ""1"")","c(""all-purpose flour"", ""salt"", ""baking powder""...",NaN,NaN,137.5,10.7,4.2,10.0,50.1,8.9,0.3,0.1,1.2,54.0,6 single pie crusts,"c(""Mix the dry ingredients."", ""Barely melt the..."
403979,418764,Gratin With Pasta and Peas,787329,zenveg,30.0,30,60,2010-04-02T09:17:00Z,Make and share this Gratin With Pasta and Peas...,character(0),One Dish Meal,"c(""Vegetable"", ""< 60 Mins"", ""Easy"")","c(""30"", ""5"", ""2"", ""1/2"", ""1/2"", ""1/2"", ""1/2"", ...","c(""butter"", ""flour"", ""milk"", ""dried basil"", ""d...",NaN,NaN,323.5,16.4,8.8,204.2,507.8,30.7,2.8,2.9,13.4,5.0,1 gratin,"c(""Melt the butter in a sauce pan at medium he..."
202464,211272,Perfect Snickerdoodles,221733,sherrilljo

## RecipeCategory

In [5]:
# # Drop 'Free Of...', 'High In...'
# recipes_cleaned = recipes[~recipes['RecipeCategory'].isin(['Free Of...', 'High In...'])]

In [6]:
# recipes_cleaned['RecipeCategory'].unique()

In [7]:
recipes_cleaned = recipes.copy()

## Keywords

In [8]:
for index, row in recipes_cleaned.iloc[3000:4000].iterrows():
    print(f"Row {index}: {row['Keywords']}")

Row 3000: c("European", "Low Cholesterol", "Healthy", "< 15 Mins", "Oven")
Row 3001: c("Breads", "Berries", "Fruit", "Healthy", "Kid Friendly", "Sweet", "Weeknight", "Oven", "< 4 Hours")
Row 3002: c("Potato", "Tropical Fruits", "Fruit", "Vegetable", "Low Protein", "Healthy", "< 60 Mins", "Easy")
Row 3003: c("Meat", "Chinese", "Asian", "< 30 Mins")
Row 3004: c("< 60 Mins", "Oven")
Row 3005: c("European", "Low Cholesterol", "Healthy", "Kid Friendly", "< 15 Mins", "Easy")
Row 3006: "Oven"
Row 3007: nan
Row 3008: c("Dessert", "European", "< 15 Mins")
Row 3009: c("< 15 Mins", "Easy")
Row 3010: c("Healthy", "< 4 Hours")
Row 3011: c("Pineapple", "Tropical Fruits", "Fruit", "< 15 Mins", "Steam")
Row 3012: "< 15 Mins"
Row 3013: "< 15 Mins"
Row 3014: c("Beef Liver", "Meat", "< 15 Mins", "Easy")
Row 3015: c("Meat", "European", "< 15 Mins", "Easy")
Row 3016: c("High In...", "< 15 Mins", "Oven")
Row 3017: nan
Row 3018: c("< 15 Mins", "Easy")
Row 3019: c("Meat", "< 4 Hours", "Easy")
Row 3020: c("Des

In [9]:
recipes_cleaned['Keywords'] = recipes_cleaned['Keywords'].apply(lambda x: x[2:-1] if isinstance(x, str) else x)

In [10]:
# Time
recipes_cleaned['Time'] = recipes_cleaned['Keywords'].str.extract(r'(\d+\s*(?:Hours|Mins))')
def convert_to_minutes(time_str):
    if pd.isna(time_str):
        return None
    match = re.match(r'(\d+)\s*(Hours|Mins)', time_str)
    if match:
        value, unit = match.groups()
        value = int(value)
        return value * 60 if unit == 'Hours' else value
    return None

recipes_cleaned['Duration'] = recipes_cleaned['Time'].apply(convert_to_minutes)

# Health
recipes_cleaned['IsHealthy'] = recipes_cleaned['Keywords'].fillna('').str.contains('Healthy').astype(int)

recipes_cleaned.drop(columns=['Time'], inplace=True)

In [11]:
# Cuisine Types
cuisine_types = [
    "African", "American", "Asian", "British", "Caribbean", "Chinese", "European", "Portuguese", "Southwestern U.S.",
    "French", "Greek", "Indian", "Italian", "Japanese", "Korean", "Russian", "Brazilian", "Southwest Asia (middle East)",
    "Mediterranean", "Mexican", "Middle Eastern", "South American", "Spanish", "Thai", "Scottish", "German", 
    "Turkish", "Vietnamese", "Tex Mex", "Hawaiian", "Moroccan", "Austrian", "Kosher", "Canadian",
    "Cajun", "Creole", "Cuban", "Dominican", "Filipino", "Finnish", "Hungarian",
    "Icelandic", "Irish", "Jamaican", "Latvian", "Malaysian", "Nigerian", "Norwegian",
    "Peruvian", "Polish", "Romanian", "Swedish", "Swiss", "Ukrainian", "Azerbaijani",
    "Georgian", "Armenian", "Lebanese", "Syrian", "Iraqi", "Iranian", "Israeli", "Jordanian",
    "Kuwaiti", "Omani", "Palestinian", "Qatari", "Saudi Arabian", "Emirati", "Yemeni", "Afghan",
    "Chadian", "Sudanese", "Egyptian", "Tunisian", "Algerian", "Moroccan", "Mauritanian",
    "Western Saharan", "Libyan", "Ghanaian", "Ivorian", "Burkinabe", "Togolese", "Beninese",
    "Nigerien", "Malian", "Guinean", "Guinean-Bissau", "Sierra Leonean", "Liberian", "Ivorian",
    "Gambian", "Senegalese", "Mauritanian", "Cape Verdean",
    "Angolan", "Cameroonian", "Central African", "Equatorial Guinean", "Gabonese", "São Toméan",
    "Antiguan and Barbudan", "Bahamian", "Barbadian", "Belizean", "Costa Rican", "Guatemalan", "Honduran",
    "Nicaraguan", "Panamanian", "Bermudian", "Saint Kitts and Nevis", "Saint Lucian", "Saint Vincent and the Grenadines",
    "Trinidadian and Tobagonian", "Vincentian", "Haitian", "Puerto Rican", "Cajun Creole", "Louisiana Creole",
    "Afro-American", "California Cuisine", "New American", "Pacific Rim", "Soul Food", "Hamburgers", "Hot Dogs",
    "Native American", "Texan", "Midwestern", "Northeastern", "Southern", "Southwestern", "Western", "British Virgin Islands",
    "Caymanian", "Cuban-American", "Dominican-American", "Puerto Rican-American", "Salvadoran",
    "West Indian", "Austrian", "Bavarian", "Berlin", "Brandenburg", "Frankfurt", "Hamburg", "Hessian", "Lower Saxon",
    "Munich", "North Rhine-Westphalian", "Palatine", "Rhineland", "Saarland", "Saxon", "Swabian", "Thuringian",
    "Upper Saxon", "Württemberg", "French Guianese", "Guyanese", "Surinamese", "Trinidadian", "Tobagonian",
]

pattern = '|'.join(cuisine_types)

# Assuming df['Keywords'] is the column you want to search for cuisine types
recipes_cleaned['Cuisine'] = recipes_cleaned['Keywords'].apply(lambda x: ', '.join([cuisine for cuisine in cuisine_types if pd.notna(x) and cuisine.lower() in x.lower()]))



In [12]:
# Free of ...
recipes_cleaned['Keywords'] = recipes_cleaned['Keywords'].fillna('')
recipes_cleaned['FreeOf'] = recipes_cleaned['Keywords'].str.contains("Free of", case=False).astype(int)

# High in
recipes_cleaned['HighIn'] = recipes_cleaned['Keywords'].str.contains("High ", case=False).astype(int)

# Low in
recipes_cleaned['LowIn'] = recipes_cleaned['Keywords'].str.contains("Low ", case=False).astype(int)

In [13]:
recipes_cleaned.head(20)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Duration,IsHealthy,Cuisine,FreeOf,HighIn,LowIn
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,1440.0,45,1485,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"""Dessert"", ""Low Protein"", ""Low Cholesterol"", ""...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan...",NaN,1,,1,0,1
1,39,Biryani,1567,elly9812,25.0,240,265,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"", ""...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and...",NaN,0,"Asian, Indian",0,1,0
2,40,Best Lemonade,1566,Stephen Little,5.0,30,35,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"""Low Protein"", ""Low Cholesterol"", ""Healthy"", ""...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ...",60.0,1,,0,0,1
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,20.0,1440,1460,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",Soy/Tofu,"""Beans"", ""Vegetable"", ""Low Cholesterol"", ""Week...","c(""12"", ""1"", ""2"", ""1"", ""10"", ""1"", ""3"", ""2"", ""2...","c(""extra firm tofu"", ""eggplant"", ""zucchini"", ""...",4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc...",NaN,0,,0,0,1
4,42,Cabbage Soup,1538,Duckie067,30.0,20,50,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"""Low Protein"", ""Vegan"", ""Low Cholesterol"", ""He...","c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil...",60.0,1,,0,0,1
5,43,Best Blackbottom Pie,34879,Barefoot Beachcomber,120.0,20,140,1999-08-21T10:35:00Z,Make and share this Best Blackbottom Pie recip...,character(0),Pie,"""Dessert"", ""Weeknight"", ""Stove Top"", ""< 4 Hours""","c(""1 1/4"", ""1/4"", ""6"", ""1/3"", ""1/4"", ""1/4"", ""2...","c(""graham cracker crumbs"", ""sugar"", ""butter"", ...",1.0,1.0,437.9,19.3,10.9,94.3,267.6,58.0,1.8,42.5,7.0,8.0,1 9-inch pie,"c(""Graham Cracker Crust: In small bowl, combin...",240.0,0,,0,0,0
6,44,Warm Chicken A La King,1596,Joan Edington,3.0,35,38,1999-09-17T04:47:00Z,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"""Poultry"", ""Meat"", ""< 60 Mins""","c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,23.0,895.5,66.8,31.9,405.8,557.2,29.1,3.1,5.0,45.3,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co...",60.0,0,,0,0,0
7,45,Buttermilk Pie With Gingersnap Crumb Crust,1580,tristitia,50.0,30,80,1999-08-06T00:40:00Z,Make and share this Buttermilk Pie With Ginger...,"""https://img.sndimg.com/f

In [14]:
recipes_cleaned.iloc[61:62]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Duration,IsHealthy,Cuisine,FreeOf,HighIn,LowIn
61,103,Borscht,1550,Linda Sundbye,30.0,20,50,1999-08-16T05:27:00Z,Make and share this Borscht recipe from Food.com.,character(0),Vegetable,"""Russian"", ""European"", ""Low Protein"", ""Low Cho...","c(""2"", ""1"", ""1"", ""2"", ""1"", ""1"", ""1"", ""1"", ""1"",...","c(""cabbage"", ""onion"", ""celery"", ""butter"", ""bee...",3.0,3.0,631.8,24.7,14.9,61.1,1759.0,100.9,18.3,70.5,14.3,NaN,1 batch,"c(""Saute cabbage, onion and celery in butter u...",60.0,1,"European, Russian",1,0,1


## Remove C()

In [15]:
columns_to_clean = ['RecipeInstructions', 'RecipeIngredientParts', 'Keywords', 'Images', 'RecipeIngredientQuantities']

for column in columns_to_clean:
    recipes_cleaned.loc[:, column] = recipes_cleaned[column].str.replace(r'c\(|"|\'|\)', '', regex=True)

## Images

In [16]:
recipes_cleaned['Num_Images'] = recipes_cleaned['Images'].str.count("https://")

In [17]:
recipes_cleaned.iloc[46663]

RecipeId                                                                  50672
Name                                                            Ranch Salsa Dip
AuthorId                                                                  30534
AuthorName                                                                Lorac
CookTime                                                                   60.0
PrepTime                                                                      5
TotalTime                                                                    65
DatePublished                                              2003-01-10T20:29:00Z
Description                   A souper creamy salsa, perfect for Souper Bowl...
Images                                                              character(0
RecipeCategory                                                        Vegetable
Keywords                      Kid Friendly, Potluck, Weeknight, No Cook, Ref...
RecipeIngredientQuantities              

_____

In [18]:
recipes_cleaned[~recipes_cleaned['RecipeIngredientQuantities'].isnull()]

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions,Duration,IsHealthy,Cuisine,FreeOf,HighIn,LowIn,Num_Images
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,1440.0,45,1485,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,https://img.sndimg.com/food/image/upload/w_555...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"Toss 2 cups berries with sugar., Let stand for...",NaN,1,,1,0,1,6.0
1,39,Biryani,1567,elly9812,25.0,240,265,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,https://img.sndimg.com/food/image/upload/w_555...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,Soak saffron in warm milk for 5 minutes and pu...,NaN,0,"Asian, Indian",0,1,0,2.0
2,40,Best Lemonade,1566,Stephen Little,5.0,30,35,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,https://img.sndimg.com/food/image/upload/w_555...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"Into a 1 quart Jar with tight fitting lid, put...",60.0,1,,0,0,1,2.0
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,20.0,1440,1460,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,https://img.sndimg.com/food/image/upload/w_555...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"Drain the tofu, carefully squeezing out excess...",NaN,0,,0,0,1,2.0
4,42,Cabbage Soup,1538,Duckie067,30.0,20,50,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,https://img.sndimg.com/food/image/upload/w_555...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"Mix everything together and bring to a boil., ...",60.0,1,,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,35.0,60,95,2020-12-22T15:27:00Z,Make and share this Meg's Fresh Ginger Gingerb...,character(0,Dessert,4 Hours,"3, 1/2, 1/2, 1/4, 1/4, 1, 1/4, 1 1/2, 1, 1/2, ...","fresh ginger, unsalted butter, dark brown suga...",NaN,NaN,316.6,12.5,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,Preheat oven to 350&deg;F Grease an 8x8 cake p...,240.0,0,,0,0,0,0.0
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,180.0,30,210,2020-12-22T15:32:00Z,"White, black, green, and pink peppercorns add ...",https://img.sndimg.com/food/image/upload/w_555...,Very Low Carbs,"High Protein, High In..., < 4 Hours","9, 2, 4, 2, 1/3, 3 1/2, 1/3","Dijon mustard, garlic, peppercorns, shallot, c...",NaN,NaN,2063.4,172.4,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,Position rack in center of oven and preheat to...,240.0,0,,0,1,0,1.0
522514,541381,Kirshwasser Ice Cream,2001131545,Jonathan F.,180.0,60,240,2020-12-22T15:33:00Z,Make and share this Kirshwasser Ice Cream reci...,character(0,Ice Cream,"Dessert, < 4 Hours","3, 3, 1/2, 1/2, 2, 2","half-and-half, heavy crea

In [19]:
recipes_cleaned.shape

(522517, 35)

In [26]:
recipes_cleaned.to_csv('recipes_extended4.csv', index=False)